In [1]:
# import the new GraphMap
%run scene_graph.ipynb

import ai2thor.controller
import numpy as np


In [167]:

class Task:
    def __init__(self):
        pass
    
    def done(self, env):
        pass
    
    def reward(self, env):
        pass
# =============================================================================
# =============================================================================
# =============================================================================

class PickTask(Task):
    def __init__(self, target_object_type='Cup'):
        self.target_object_type = target_object_type
    # =========================================================================
    def reward(self, env):
        event = controller.step(action='Pass')
        for obj in [o for o in event.metadata['objects'] if o['visible']]:
            if obj['objectType'] == self.target_object_type:
                return 1
        return -0.1
    # =========================================================================
    def done(self, env):
        if self.reward(env)==1:
            return True
        else:
            return False
    # =========================================================================
    def is_possible(self, env):
        event = controller.step(action='Pass')
        targets = [obj for obj in event.metadata['objects'] if obj['objectType']==self.target_object_type]        
        if len(targets) == 0:
            return False, targets
        else:
            return True, targets
        

In [2]:
class ThorEnvironment:
    def __init__(self, scene='FloorPlan29', grid_size=0.25, controller=None, word_vec=None, one_hot=False):                
                
        # initialise the AI2Thor controller
        if controller is None:
            self.controller = ai2thor.controller.Controller(scene=scene, gridSize=grid_size, visibilityDistance=0.5)                
        else:
            self.controller = controller
                                   
        # scene graph
        self.graph = SceneGraph(word_vec, one_hot)         
        
        self.verbose = True
        self.last_pose = None
        
    # ===============================================================================================        
    def hide_objects(self):
        event = self.controller.step(dict(action='Pass'))

        # remove all objects from those receptacles
        sourceTypes = ['CounterTop', 'Sink']
        sources = [obj for obj in event.metadata['objects'] if obj['objectType'] in sourceTypes]


        # those are the receiving receptacles we will put objects to
        sinkTypes = ['Cabinet']
        sinks = [obj for obj in event.metadata['objects'] if obj['objectType'] in sinkTypes]

        for src in sources:
            print('Clearing', src['objectId'])
            for obj in src['receptacleObjectIds']:
                sink = np.random.choice(sinks)
                print('\tattempting to move', obj, 'to', sink['objectId'])
                event = self.controller.step(dict(action = 'GetSpawnCoordinatesAboveReceptacle', objectId=sink['objectId'], anywhere=True))
                for pos in event.metadata['actionReturn']:
                    event = self.controller.step(dict(action='PlaceObjectAtPoint', objectId=obj, position=pos))
                    if event.metadata['lastActionSuccess']:                    
                        break            

        # check we were successful
        sourcesIds = [s['objectId'] for s in sources]    
        event = self.controller.step(dict(action='Pass'))
        for obj in event.metadata['objects']:        
            if obj['parentReceptacles'] is not None:
                for p in obj['parentReceptacles']:                
                    if p in sourcesIds:
                        return False
        return True
        
    # ===============================================================================================        
    def update_object(self, obj, pose=None):
        
        # does the object exist in the graph?
        idx = self.graph.find('name', obj['name']) 
        
        # if not, add it and its affordances
        if idx == []:                        
            idx = self.graph.add_object(obj['objectType'].lower(), data=obj)                
        else:
            # if the object exists, update it with the observation given in obj
            assert(len(idx)==1)
            idx = idx[0]
            self.graph.nodes[idx]['data'] = obj
            
        # if pose is given, check that the object has a goto affordance with that pose connected to it
        if pose is not None:                        
            if len([g for g in self.graph.get_affordances(idx, 'go') if self.graph.nodes[g]['data']['pose']==pose])==0:
                self.graph.add_affordance(idx, 'go', data={'pose':pose})                    
        
        # add other affordances as neccessary
        if obj['pickupable'] and not self.graph.has_affordance(idx, 'pick'):
            self.graph.add_affordance(idx,'pick')
        
        if obj['receptacle'] and not self.graph.has_affordance(idx, 'put'):
            self.graph.add_affordance(idx,'put')
            
        if obj['openable'] and not self.graph.has_affordance(idx, 'open'):
            self.graph.add_affordance(idx,'open')
            
        if obj['openable'] and not self.graph.has_affordance(idx, 'close'):
            self.graph.add_affordance(idx,'close')
        
        # New
        if obj['sliceable'] and not self.graph.has_affordance(idx, 'slice'):
            self.graph.add_affordance(idx,'slice')

        if obj['cookable'] and not self.graph.has_affordance(idx, 'cook'):
            self.graph.add_affordance(idx,'cook')
        
        if obj['breakable'] and not self.graph.has_affordance(idx, 'break'):
            self.graph.add_affordance(idx,'break')

        if obj['dirtyable'] and not self.graph.has_affordance(idx, 'dirty'):
            self.graph.add_affordance(idx,'dirty')
            
        if obj['dirtyable'] and not self.graph.has_affordance(idx, 'clean'):
            self.graph.add_affordance(idx,'clean')

        if obj['canBeUsedUp'] and not self.graph.has_affordance(idx, 'use_up'):
            self.graph.add_affordance(idx,'use_up')
        
        # TODO: Heat and Cool
        
#         if obj['toggleable'] and not self.graph.has_affordance(idx, 'toggle'):
#             self.graph.add_affordance(idx,'toggle')            
        
        # if the object is a receptable, check if its children are already in the map and make sure there is an edge between them
        if obj['receptacleObjectIds'] is not None:
            for child in obj['receptacleObjectIds']:            
                try:
                    # for the children that are in the map already
                    child_idx = self.graph.find('objectId', child)[0]                                

                    # is there an edge from parent to child?
                    if not child_idx in self.graph.successors(idx):
                        self.graph.add_relation_edge(idx, child_idx, 'contains')

                    # is there an edge from child to parent?
                    if not idx in self.graph.successors(child_idx):
                        self.graph.add_relation_edge(child_idx, idx, 'in')

                except IndexError:   
                    # the child object is not yet in the map, we can't do anything about it now
                    pass
                                                        
        
        # if the object is inside a receptable, check if the receptable is already in the map and make sure there is an edge between them
        if obj['parentReceptacles'] is not None:
            for parent in obj['parentReceptacles']:
                try:
                    # for the parents that are in the map already
                    parent_idx = self.graph.find('objectId', parent)[0]              
                    
                    # is there an edge from parent to child?
                    if not idx in self.graph.successors(parent_idx):
                        self.graph.add_relation_edge(parent_idx, idx, 'contains')

                    # is there an edge from child to parent?
                    if not parent_idx in self.graph.successors(idx):
                        self.graph.add_relation_edge(idx, parent_idx, 'in')
                except IndexError:   
                    pass
                    # the parent object is not yet in the map, we can't do anything about it now
                    
        
    # ===============================================================================================        
    def explore_environment(self):
                    
        self.graph.clear()
        
        event = self.controller.step(dict(action='LookDown', degrees=20))            
        # get all valid positions        
        event = self.controller.step(dict(action='GetReachablePositions'))
        reachable = event.metadata['reachablePositions']
        
        # teleport the robot to all positions, then rotate in place by a fixed angle and remember which objects are visible
        for loc in reachable:
            action = dict(action='Teleport')
            action.update(loc)    
            self.controller.step(action)

            for rot in range(0,360,45):   # rotate in 45 deg increments
                event = self.controller.step(dict(action='Rotate', rotation=rot))
                pose = [loc['x'], loc['y'], loc['z'], rot]                                                
                self.last_pose = pose
                
                for obj in [o for o in event.metadata['objects'] if o['visible']]:
                    self.update_object(obj, pose)

        self.graph.to_torch_graph()
    # ===============================================================================================
    def step(self, idx):
        assert self.graph.nodes[idx]['node_type'] == 'affordance', 'Not an affordance: %d' % idx
        
        if self.graph.nodes[idx]['affordance'] == 'go':            
            # Teleport the agent to the new pose     
            pose = self.graph.nodes[idx]['data']['pose']
            event = controller.step(dict(action='TeleportFull', x=pose[0], y=pose[1], z=pose[2], rotation=pose[3], horizon=20))
            
            # was this successful?
            if event.metadata['lastActionSuccess']:
                self.last_pose = pose
                
                # we have to remove the 'at' relations between the robot and the pose affordance and its object                
                for node in self.graph.get_relations(self.graph.robot_node, 'at'):
                    self.graph.remove_edge(self.graph.robot_node, node)
            
                # insert a new 'at' relation between robot and new pose affordance, and the object of that affordance                
                obj = next(self.graph.predecessors(idx))     
                assert len(list(self.graph.predecessors(idx)))==1
                
                self.graph.add_relation_edge(self.graph.robot_node, obj, 'at')
                self.graph.add_relation_edge(self.graph.robot_node, idx, 'at')
                
        # +++++++++    
        elif self.graph.nodes[idx]['affordance'] == 'pick':      
            # which object are we trying to pick?
            obj = next(self.graph.predecessors(idx))
            objectId = self.graph.nodes[obj]['data']['objectId']
            event = controller.step(dict(action='PickupObject', objectId=objectId))
                        
            # was this successful?
            if event.metadata['lastActionSuccess']:                      
                # if yes, we have to remove the edges to the affordances
                self.graph.remove_nodes_from(self.graph.get_affordances(obj))
                                
                # remove relation edges (from this object to other objects)
                succ, pred = self.graph.get_related_objects(obj)
                for s in succ:
                    self.graph.remove_edge(obj, s)
                for p in pred:
                    self.graph.remove_edge(p, obj)                                    
                                
                # add new relation edges to the robot node
                self.graph.add_relation_edge(self.graph.robot_node, obj, 'contains')
                self.graph.add_relation_edge(obj, self.graph.robot_node, 'in')
                                
                self.graph.to_torch_graph()
                                
        # +++++++++         
        elif self.graph.nodes[idx]['affordance'] == 'put':   
            # which object are we attempting to put down?
            inhand = self.graph.get_relations(self.graph.robot_node, 'contains')
            
            if len(inhand)<1:
                if self.verbose:
                    print('! Error when executing PUT affordance: Agent does not carry an object.', inhand)
                return False
            elif len(inhand)>1:
                if self.verbose:
                    print('! Error when executing PUT affordance: Agent carries multiple objects.')
                    print(inhand, [self.graph.nodes[x]['data']['name'] for x in inhand])                
                return False
            else:
                inhand=inhand[0]
            
            # make sure we have an object in our hand!
            if self.graph.nodes[inhand]['node_type'] != 'object':
                if self.verbose:
                    print('! Error when executing PUT affordance: Not an object.', obj, self.graph)
                return False
            
            # which object are we trying to put something on?
            target = next(self.graph.predecessors(idx))
            objectId_target = self.graph.nodes[target]['data']['objectId']           
            
            # get spawn coordinates
            event = controller.step(dict(action='GetSpawnCoordinatesAboveReceptacle', objectId=objectId_target, anywhere=False))
            spawn = event.metadata['actionReturn']
            
            if spawn == [] or spawn is None:
                if self.verbose:
                    print('! Error when executing PUT affordance. No spawn points on', self.graph.nodes[target]['data']['name'])                    
                return False
            
            objectId_inhand = self.graph.nodes[inhand]['data']['objectId']   
            for spawn_point in spawn:            
                event = controller.step(dict(action='PlaceObjectAtPoint', objectId=objectId_inhand, position=spawn_point))
                
                if event.metadata['lastActionSuccess']:
                    break
      
            # was this successful?
            if event.metadata['lastActionSuccess']:
               
                controller.step(dict(action='DropHandObject'))
                event = controller.step(dict(action='PlaceObjectAtPoint', objectId=objectId_inhand, position=spawn_point))
                
                # remove connection between robot and dropped inhand object
                self.graph.remove_edge(inhand, self.graph.robot_node)
                self.graph.remove_edge(self.graph.robot_node, inhand)
                
                # we need to update both the target and the dropped object
                obj = [o for o in event.metadata['objects'] if o['name']==self.graph.nodes[inhand]['data']['name']][0]   
                self.update_object(obj, pose = self.last_pose)
                
                obj = [o for o in event.metadata['objects'] if o['name']==self.graph.nodes[target]['data']['name']][0]
                self.update_object(obj)
            else:
                if self.verbose:
                    print(event.metadata['lastAction'], event.metadata['lastActionSuccess'], event.metadata['errorMessage'])   
                                                                             
        # +++++++++       
        elif self.graph.nodes[idx]['affordance'] == 'open':    
            # which object are we trying to open?
            obj = next(self.graph.predecessors(idx))
            objectId = self.graph.nodes[obj]['data']['objectId']
            event = self.controller.step(dict(action='OpenObject', objectId=objectId))                        
        # +++++++++       
        elif self.graph.nodes[idx]['affordance'] == 'close':                 
            # which object are we trying to close?
            obj = next(self.graph.predecessors(idx))
            objectId = self.graph.nodes[obj]['data']['objectId']
            event = self.controller.step(dict(action='CloseObject', objectId=objectId))   
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'slice':                 
            # which object are we trying to slice?
            obj = next(self.graph.predecessors(idx))
            objectId = self.graph.nodes[obj]['data']['objectId']
            event = self.controller.step(dict(action='SliceObject', objectId=objectId))  
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'heat':                 
            # which object are we trying to heat?
            obj = next(self.graph.predecessors(idx))
            objectId = self.graph.nodes[obj]['data']['objectId']
            event = self.controller.step(dict(action='HeatObject', objectId=objectId))  
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'cool':                 
            # which object are we trying to cool?
            obj = next(self.graph.predecessors(idx))
            objectId = self.graph.nodes[obj]['data']['objectId']
            event = self.controller.step(dict(action='CoolObject', objectId=objectId))  
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'clean':                 
            # which object are we trying to clean?
            obj = next(self.graph.predecessors(idx))
            objectId = self.graph.nodes[obj]['data']['objectId']
            event = self.controller.step(dict(action='CleanObject', objectId=objectId)) 
        # +++++++++       
        elif self.graph.nodes[idx]['affordance'] == 'toggle':                 
            raise NotImplementedError        
                        
        # robot affordances (moving, looking up and down, crouching, standing)
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'crouch':                 
            event = self.controller.step(dict(action='Crouch'))             
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'stand':                 
            event = self.controller.step(dict(action='Stand'))             
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'up':                 
            event = self.controller.step(dict(action='LookUp'))          
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'down':                 
            event = self.controller.step(dict(action='LookDown'))                  
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'left':                 
            event = self.controller.step(dict(action='MoveLeft'))          
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'right':                 
            event = self.controller.step(dict(action='MoveRight'))                   
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'ahead':                 
            event = self.controller.step(dict(action='MoveAhead'))            
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'back':                 
            event = self.controller.step(dict(action='MoveBack'))           
        # +++++++++
        elif self.graph.nodes[idx]['affordance'] == 'done':                 
            raise NotImplementedError('Done affordance was called.')
            
        else:
            raise NotImplementedError('Unknown affordance', self.graph.nodes[idx]['affordance'] )
                        
        if event.metadata['lastActionSuccess']:
            result = True
        else:
            if self.verbose:
                print(event.metadata['lastAction'], event.metadata['lastActionSuccess'], event.metadata['errorMessage'])                
            result = False       
            
        
        # update visible objects (except the picked up one, if any)
        for obj in [o for o in event.metadata['objects'] if o['visible'] and not o['isPickedUp']]:
            self.update_object(obj, self.last_pose)                    
        
        return result
            
#     # ===============================================================================================          
#     def add_dynamic_objects(self):
#         event = controller.step(action='Pass')
       
#         # find non-static objects
#         for obj in [o for o in event.metadata['objects'] if o['visible'] and o['pickupable']]:
#             name = obj['objectId']
#             pose = [obj['position']['x'], obj['position']['y'], obj['position']['z'], 0]                        
#             self.current_dynamic_objects.append(name)
            
#             landmark = self.graph.add_landmark(pose=pose, name=name, data=obj, repr=self.word_vec.get_vecs_by_tokens(obj['objectType'].lower()))                
#             self.graph.add_edge((self.graph.node(self.current_pose).name, name), undirected=True)
    
#         # return how many dynamic objects were visible
#         return len(self.current_dynamic_objects)
    
#     # ===============================================================================================          
#     def remove_dynamic_objects(self):        
#         result = len(self.current_dynamic_objects)
#         while len(self.current_dynamic_objects) > 0:
#             name = self.current_dynamic_objects.pop()
#             self.graph.remove_landmark(name)
        
#         return result
            
#     # ===============================================================================================          
#     def step(self, affordance):
        
#         if not 'affordance' in affordance.data.keys():
#             print('Not an affordance!')
#             return None
        
#         # remove all dynamic objects from the graph visible before executing the action
#         removed = self.remove_dynamic_objects()
        
#         # We want to move to a new pose.   
#         if affordance.data['affordance'] == 'go':                       
            
#             # Which pose node is the affordance connected to?   
#             goto_pose_id = self.graph.connected_nodes(affordance.id)[0] 
#             pose = self.graph.node(goto_pose_id).pose
            
#             # Teleport the agent to the new pose            
#             event = controller.step(dict(action='TeleportFull', x=pose[0], y=pose[1], z=pose[2], rotation=pose[3], horizon=0))
#             self.current_pose = goto_pose_id
                           
#         elif affordance.data['affordance'] == 'open':
#             object_id = self.graph.connected_nodes(affordance.id)[0] 
#             event = controller.step(action='OpenObject', objectId=self.graph.node(object_id).name)
# #             print('Open', self.graph.node(object_id).name)
        
#         elif affordance.data['affordance'] == 'close':
#             object_id = self.graph.connected_nodes(affordance.id)[0] 
#             event = controller.step(action='CloseObject', objectId=self.graph.node(object_id).name)
# #             print('Close', self.graph.node(object_id).name)
#         else:
#             pass
# #             print(affordance.data)
            
#         # add dynamic object visible after executing the action
#         added = self.add_dynamic_objects()
            
#         # re-build the torch graph if we have to
#         if (removed>0 or added>0):
#             self.graph.update_torch_graph()
#             # self.graph.construct_torch_graph()

In [143]:
# controller.reset('FloorPlan29')
# env = ThorEnvironment(controller=controller, word_vec=word_vec)
# env.explore_environment()



In [163]:
# # env.explore_environment()
# task = PickTask()
# # env.step(env.graph.node('a12'))
# print(env.current_dynamic_objects)
# print('Reward: ', task.reward(env))

6
['Potato|+00.28|+01.09|+00.12', 'Fork|+00.03|+01.03|+00.12', 'Kettle|+00.01|+01.05|-01.40', 'ButterKnife|+00.11|+01.03|+00.37', 'Bread|+00.41|+01.11|+00.35', 'Spoon|+00.07|+01.03|+00.23']
Reward:  0


KeyError: 'affordance'

NameError: name 'env' is not defined